<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/122_Logging_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Logging** is the *next essential skill* for building reliable, debuggable agents. You're about to level up your agent-building toolkit significantly.

---

## 🧾 What Is Logging?

**Logging** means recording what your code is doing — especially:

* What happened
* When it happened
* Where it happened
* What went wrong (if anything)

Python provides this via the built-in [`logging`](https://docs.python.org/3/library/logging.html) module, which is:

* More powerful and flexible than `print()`
* Meant for debugging, monitoring, and tracing
* Used across all serious Python software (not just agents)

---

## 🧠 Why Is Logging Important?

### 1. **Debugging**

Logging helps you *see what happened* when something fails — especially after the fact.

### 2. **Monitoring**

In deployed agents (server, service, background job), you don't see stdout — but logs can tell you what the agent did, step by step.

### 3. **Error Visibility**

Logging exceptions (especially with tracebacks) helps you catch and fix bugs **without needing to reproduce them live**.

### 4. **Postmortems**

You can review logs to understand:

* Where a failure happened
* How often something breaks
* Whether retries succeeded

---

## 🤖 Why Logging Is Critical for Agents

Agents are:

* Complex
* Long-running
* Unpredictable (due to external APIs, LLMs, tools)

Which makes them **very hard to debug** unless you have logs!

So in agent systems, logging helps with:

* **Step-by-step traces** of decisions
* Tracking **tool calls, retries, failures**
* Recording **inputs/outputs** for reproducibility
* Logging **LLM outputs** for audit and debugging
* Providing structured logs you can search/index

---

## 🧱 How Logging Works in Python

Here's a minimal example:

```python
import logging

logging.basicConfig(level=logging.INFO)

logging.info("Agent started")
logging.warning("Low disk space")
logging.error("Tool failed")
```

You get:

```
INFO:root:Agent started
WARNING:root:Low disk space
ERROR:root:Tool failed
```

---

## 🧨 Logging Exceptions

In your `except` block, use `logging.exception()`:

```python
try:
    risky()
except Exception as e:
    logging.exception("risky() failed")
```

This gives you:

```
ERROR:root:risky() failed
Traceback (most recent call last):
  ...
ZeroDivisionError: division by zero
```

✅ Includes the traceback
✅ You don’t need to format it yourself
✅ Works great with tools like `run_agent_step`

---

## 🛠️ Logging in Agent Error Wrappers

Remember this pattern?

```python
try:
    ...
except RetryableToolError as e:
    logging.warning("Retryable error: %s", e, exc_info=True)
```

That `exc_info=True` includes the traceback.

So you log:

* The fact that something failed
* Whether it’s retryable
* The actual traceback (optional but ideal)

---

## 🧰 Agent-Specific Logging Best Practices

| Best Practice                           | Why                                                    |
| --------------------------------------- | ------------------------------------------------------ |
| Use `logging` (not `print`)             | Works in background jobs, saves to files, configurable |
| Log each **tool call**                  | Helps track what tools the agent uses                  |
| Log each **tool result**                | Helps debug when outputs are malformed                 |
| Log each **error with traceback**       | So you can reproduce or fix failures                   |
| Include **timing info** (`time.time()`) | For slow steps or performance tuning                   |
| Optional: Log inputs/outputs            | For reproducibility (if data isn't sensitive)          |

---

## 🔚 Summary: Logging for Agent Builders

✅ **What**: Logging records what your code does
✅ **Why**: Essential for debugging, monitoring, reliability
✅ **How**: Use `logging.info`, `logging.error`, `logging.exception`
✅ **In agents**: Log every tool call, response, failure, retry, and decision



You're developing **agent forensics** — using timestamps like clues at a crime scene.

---

## 🕰️ Why Time Matters in Logging

Timestamps let you:

### ✅ 1. **Reconstruct what happened**

Logs become a **narrative**:

```
12:00:01 Tool A called with x
12:00:03 Tool A returned y
12:00:03 Tool B called with y
12:00:05 Tool B failed: KeyError
```

You now know:

* Tools were called in order
* Tool B failed after receiving output from Tool A
* The failure likely traces back to something wrong with `y`

---

### ✅ 2. **Identify slow steps**

Example:

```
12:00:01 Calling API...
12:00:10 API responded
```

→ That API call took **9 seconds**, which might be too long. You could:

* Add a timeout
* Retry faster
* Switch APIs

---

### ✅ 3. **Detect retries, loops, flapping behavior**

If you see this:

```
12:00:01 Tool failed (timeout)
12:00:02 Retrying...
12:00:03 Tool failed (timeout)
12:00:04 Retrying...
```

You know the agent is stuck in a **retry loop**.

---

### ✅ 4. **Compare step timing across runs**

By measuring **elapsed time per step**, you can:

* Optimize slow parts
* Spot regressions
* Prioritize bottlenecks

```python
t0 = time.time()
tool()
logging.info("Tool took %.2fs", time.time() - t0)
```

---

## 🧪 Pro Tip: Use both absolute and relative time

| Log Format            | Use For                      |
| --------------------- | ---------------------------- |
| `2025-09-07 12:00:01` | Real-world timestamp         |
| `tool took 2.45s`     | Step duration (perf insight) |

---

## 🧠 In Summary

Yes — time is your **storyline**.
It helps you answer:

* What happened?
* In what order?
* How long did each step take?
* Where did things go wrong?

That’s **core to debugging agents**.




## ✅ Example 1: Basic Logging with Timestamps

## 🧱 Step-by-Step Breakdown

```python
import logging
```

This imports Python’s built-in `logging` module — a powerful tool for writing messages about what your code is doing.

---

### 🛠️ `basicConfig` — Logging Setup

```python
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)
```

This is the **standard setup** for logging. It tells the logger how to behave.

#### 🧩 `level=logging.INFO`

This sets the **minimum severity of messages to show**.
Python logging has levels:

| Level      | Meaning                       | Use For                                |
| ---------- | ----------------------------- | -------------------------------------- |
| `DEBUG`    | Very detailed info (dev only) | Variable values, internal flow         |
| `INFO`     | General events (default use)  | Tool started, tool ended, user request |
| `WARNING`  | Something odd, not fatal      | Slow response, unexpected format       |
| `ERROR`    | An error occurred             | Failed API call, invalid input         |
| `CRITICAL` | Really bad (app may crash)    | Out of memory, system failure          |

🔧 You can change this to `DEBUG` if you want **more detailed output**:

```python
level=logging.DEBUG
```

---

#### 🎨 `format="%(asctime)s [%(levelname)s] %(message)s"`

This controls **how each line looks**. You can customize it with special codes:

| Format Code     | What It Shows                 |
| --------------- | ----------------------------- |
| `%(asctime)s`   | Timestamp (when it happened)  |
| `%(levelname)s` | Level (`INFO`, `DEBUG`, etc.) |
| `%(message)s`   | Your log message              |

🔎 Example output:

```
2025-09-07 15:13:24,123 [INFO] Agent step started
```

---

### 🤖 Logging Agent Events

```python
logging.info("Agent step started")
```

This logs a **normal event** — something worth recording but not a problem.

```python
x = 42
logging.debug(f"x = {x}")
```

This logs a **debug message**, which will NOT show unless you set `level=logging.DEBUG`.

```python
logging.info("Agent step completed")
```

Another normal log message — now we know this step ran end-to-end.

---

## 📺 How Do You View the Logs?

* If you’re running Python locally: they’ll appear in the **terminal / console**
* In **Colab** or **Jupyter**, they appear in the notebook output cells
* In production: logs are typically written to **files**, dashboards, or **cloud log collectors**

---

## 🧪 Optional Configurations

Yes — `basicConfig` has many more options:

| Option               | What it does                           |
| -------------------- | -------------------------------------- |
| `filename="app.log"` | Logs to a file instead of screen       |
| `filemode="w"`       | Overwrites file each time (vs. append) |
| `handlers=[...]`     | Use multiple outputs (file + console)  |
| `datefmt="%H:%M:%S"` | Custom time format                     |
| `style="{"`          | Use `{}` instead of `%` for formatting |

Example: log to a file with custom format

```python
logging.basicConfig(
    filename="agent.log",
    level=logging.DEBUG,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S"
)
```

---

## ✅ Summary

| Concept             | Why it matters                                       |
| ------------------- | ---------------------------------------------------- |
| `level=...`         | Controls **what gets shown** (filtering by severity) |
| `format=...`        | Controls **how each log looks**                      |
| `info()`, `debug()` | Used to log different kinds of messages              |
| `basicConfig(...)`  | Sets it all up with defaults                         |




In [6]:
import logging

# Setup basic logging format
logging.basicConfig(
    # level=logging.INFO,
    level=logging.DEBUG,
    format="%(asctime)s [%(levelname)s] %(message)s",
)

# Simulate an agent step
logging.info("Agent step started")

x = 42
logging.debug(f"x = {x}")

logging.info("Agent step completed")


Yes! This is a known quirk in **Google Colab** — the `logging.basicConfig(...)` **does not always behave as expected** (especially if it's been called before in the same session). But we can fix that.

---

### ✅ Fix for Logging in Google Colab

You need to **manually configure a handler**, because `basicConfig` only works the *first time it's called* in a session. Here's a working setup:

In [7]:
import logging

# Remove any existing handlers (Colab sometimes adds one)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set up logging manually
handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s [%(levelname)s] %(message)s')
handler.setFormatter(formatter)

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)

# Now try logging
logging.info("Agent step started")

x = 42
logging.debug(f"x = {x}")

logging.info("Agent step completed")


2025-09-08 18:10:46,507 [INFO] Agent step started
2025-09-08 18:10:46,510 [DEBUG] x = 42
2025-09-08 18:10:46,511 [INFO] Agent step completed


In **production-grade software**, it's common to log everything to a **file** so you can:

* Inspect logs later (debugging or audits)
* Monitor behavior in real-time (via dashboards)
* Store logs persistently across runs

---

### ✅ Logging to a File

Here’s how you’d modify the logging setup to write to a file instead of just showing output in Colab or terminal:

### 🔧 Summary of Logging Destinations

| Destination | Setup                           | Use Case                                   |
| ----------- | ------------------------------- | ------------------------------------------ |
| **Console** | `StreamHandler` (default)       | Dev/test in notebooks or CLI               |
| **File**    | `filename=...` in `basicConfig` | Persistent logs, debugging, postmortems    |
| **Both**    | Use multiple handlers           | Dev + long-term storage (advanced setup)   |
| **Remote**  | Via services like Logstash      | Production monitoring, distributed logging |


In [8]:
import logging

# Clear previous handlers if in an interactive environment like Colab
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set up file logging
logging.basicConfig(
    filename="agent.log",              # <- writes logs here
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    filemode="w",                      # "w" = overwrite each run; use "a" to append
)

# Test it
logging.info("Agent started")
logging.warning("This is a warning")
logging.error("Something went wrong!")


In [9]:
# View the contents of the log file
with open("agent.log") as f:
    print(f.read())

2025-09-08 18:12:06,683 [INFO] Agent started
2025-09-08 18:12:06,683 [WARNING] This is a warning
2025-09-08 18:12:06,683 [ERROR] Something went wrong!



**Saving logs to a file** is absolutely recommended for your remaining examples, especially since:

---

### ✅ Benefits of Saving Logs to a File

1. **Persistent History**
   Logs won’t disappear when the cell finishes running or if Colab restarts.

2. **Full Traceability**
   You can scroll back through the full run — even after multiple tool calls or retries.

3. **Final Review**
   At the end, you can **print the log file** to review what happened in sequence — very useful for debugging and learning.

4. **Production Mindset**
   This mirrors how real-world agents and software systems manage logs — preparing you for production environments.

---

### 🔧 Setup (We’ll use this moving forward)

Let’s configure logging so every example writes to the same file, and we can inspect it as needed:

---

### 🧠 Usage Flow

1. Run your code blocks (each logs to `agent_log.txt`)
2. Call `show_logs()` to view everything so far
3. At the end, you'll have a **clean, chronological trace** of your agent's run




In [10]:
import logging

# Clear previous handlers (necessary for Colab)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Configure logging to file
logging.basicConfig(
    filename="agent_log.txt",          # All logs go here
    level=logging.DEBUG,               # Capture DEBUG and above
    format="%(asctime)s [%(levelname)s] %(message)s",
    filemode="w",                      # Overwrite each run
)

def show_logs():
    with open("agent_log.txt") as f:
        print(f.read())


## ✅ Example 2: **Timing a Step (Duration Logging)**

#### 🔍 What you learn:

* How to time execution
* Why duration logs are useful (to debug slow steps)



In [11]:
import time
import logging

# Clear previous handlers (for Colab)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Setup: log to file instead of just console
logging.basicConfig(
    filename="agent_log.txt",     # Where to save logs
    level=logging.INFO,           # What severity to log (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format="%(asctime)s [%(levelname)s] %(message)s",  # How logs appear
    filemode="a"                  # "a" = append, "w" = overwrite
)


# Simulated tool call
def simulate_tool():
    time.sleep(2)
    return "result"

# Logging
logging.info("Calling tool...")
start = time.time()

result = simulate_tool()

duration = time.time() - start
logging.info(f"Tool returned: {result}")
logging.info(f"Tool call took {duration:.2f}s")

In [12]:
# 📄 To View the Logs Afterward:
def show_logs():
    with open("agent_log.txt") as f:
        print(f.read())

show_logs()

2025-09-08 18:16:36,442 [INFO] Calling tool...
2025-09-08 18:16:38,444 [INFO] Tool returned: result
2025-09-08 18:16:38,444 [INFO] Tool call took 2.00s





### 💤 `time.sleep(2)` — What It Does

This line:

```python
time.sleep(2)
```

**Pauses the program for 2 seconds**.

It’s part of Python’s built-in `time` module and is used to simulate:

* **delays** (like waiting for a network call)
* **throttling** (e.g., rate-limiting API usage)
* **time-based retry logic** (e.g., exponential backoff)
* or simply slowing things down so you can **see the timing and logging clearly**.

---

### 🧠 In Agent Context

When building agents, you might use `sleep()` to:

| Use Case                      | Example                                        |
| ----------------------------- | ---------------------------------------------- |
| 🔄 **Backoff**                | Wait longer between retries after failure      |
| 🕸 **Simulate network delay** | Mimic latency from external APIs               |
| 🚦 **Rate-limiting**          | Pause to avoid hitting external service limits |
| 🧪 **Test duration logging**  | See if logs capture realistic timing info      |

---

### 🔍 Try This:

Update your log message to show before and after the pause:

```python
logging.info("About to call the tool (simulated delay)")
time.sleep(2)
logging.info("Done waiting")
```

Then run `show_logs()` to see exactly how that 2-second pause shows up in the timestamps.




Let’s dig into how logging setup works, how flexible it is, and how you can **standardize it with a function** — just like a professional developer or agent framework would.

---

## ✅ 1. Logging Setup: How It Works

It’s very **configurable**.

You can:

* Change **where logs go**: file, console, or both.
* Choose **how they look**: include timestamps, file names, line numbers, etc.
* Set **levels** globally or per logger/module.
* Add **handlers** for email alerts, rotating files, cloud logging, etc.

---

## 🔁 2. Recommended: Wrap Logging Setup in a Function

```python
def setup_logging(log_file="agent_log.txt", level=logging.INFO):
    """Standard logging setup for agent steps."""
    import logging

    # Remove default handlers (important for notebooks like Colab)
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        filename=log_file,
        level=level,
        format="%(asctime)s [%(levelname)s] %(message)s",
        filemode="a"  # or "w" to overwrite each time
    )
```

Then just call it once at the top of your notebook:

```python
setup_logging()
```

---

## 🧠 3. Why You Should Do This in Agent Dev

| Benefit             | Why It Matters                               |
| ------------------- | -------------------------------------------- |
| 🧼 Clean separation | Keeps config out of your main logic          |
| 🔁 Reuse everywhere | Use same setup in every project / notebook   |
| 🚨 Easy to upgrade  | Add email/Slack/cloud logging later          |
| 🤖 Agent monitoring | Helps track failures, retries, latency, etc. |

---

## 🚀 4. Next-Level Logging Setups

If you're building bigger systems, you might want to:

* Log to **both console + file** (using `logging.StreamHandler`)
* Rotate logs with **`RotatingFileHandler`**
* Tag logs by **agent step**, **tool name**, or **task ID**
* Filter or format logs based on severity






## ✅ Example 3: **Logging an Error with a Traceback**


#### 🔍 What you learn:

* `logging.exception()` auto-logs the traceback
* You get both **error type + message + traceback**
* No manual `traceback.print_exc()` needed

In [5]:
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

def risky():
    return int("not_a_number")

try:
    risky()
except Exception as e:
    logging.exception("Tool failed with error")


ERROR:root:Tool failed with error
Traceback (most recent call last):
  File "/tmp/ipython-input-1357564444.py", line 9, in <cell line: 0>
    risky()
  File "/tmp/ipython-input-1357564444.py", line 6, in risky
    return int("not_a_number")
           ^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: 'not_a_number'
